In [ ]:
# Entendendo a geração de próximos tokens, diferenças devices, e lembrando o que foi dito

In [ ]:
# vendo se temos GPU disponível

# !nvidia-smi <-- não vai funcionar se não tiver GPU!

import time
import numpy as np
import torch
import transformers

print(torch.cuda.is_available()) # retorna True se encontrou uma GPU para mandarmos jobs

# dinamicamente identificando o dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

False
cpu


In [ ]:
tiny_llama = transformers.pipeline(
    task='text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device=device
)

print(tiny_llama)
print(tiny_llama.model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [ ]:
prompt = "Today is a beautiful day to"

# Maneira mais direta possivel de usar um modelo. Ele vai usar os parametros padroes para gerar a saida
output = tiny_llama(prompt, max_new_tokens=25)

# Se nao especificarmos o maximo de novos tokens, ele vai gerar ate encontrar um token que sinaliza
# fim da sentenca

print(output)
print(output[0]['generated_text']) # --> vamos tacar aqui https://platform.openai.com/tokenizer e ver como o GPT faria a tokenizacao
# Ele encontrou o mesmo numero de tokens que limitamos a geracao

[{'generated_text': 'Today is a beautiful day to be alive.\n\nVerse 2:\nThe sun is shining bright, the birds are singing,\nThe'}]
Today is a beautiful day to be alive.

Verse 2:
The sun is shining bright, the birds are singing,
The


In [ ]:
# vendo token a token o processo de geracao

# tensor eh o array que redes neurais utilizam. pt --> pytorch. tf--> tensorflow.
# soh o tensor tem o metodo to(device), que serve para mover o array para o mesmo
# dispositivo que o nosso modelo esta
prompt_input_ids = tiny_llama.tokenizer.encode(prompt, return_tensors='pt').to(device)
print(prompt_input_ids)

# para INSPECIONAR a geracao de tokens, precisamos gerar o output com novos parametros
output = tiny_llama.model.generate(
    prompt_input_ids, max_new_tokens=25,
    return_dict_in_generate=True, # retorne um dicionario com informacoes da geracao de dados
    output_scores=True, # retorne as pontuacoes de cada token gerado
)

print(output.keys())

tensor([[    1, 20628,   338,   263,  9560,  2462,   304]])
odict_keys(['sequences', 'scores', 'past_key_values'])


In [ ]:
# vamos pegar os TRANSITION SCORES (probabilidades da palavra utilizada)

transitions = tiny_llama.model.compute_transition_scores(
    output.sequences, output.scores, normalize_logits=True
)

transitions

tensor([[-1.5247e+00, -5.8344e-01, -9.5884e-01, -1.5707e+00, -1.4555e+00,
         -1.5399e+00, -9.6954e-04, -2.2908e-02, -1.0781e+00, -6.2057e-02,
         -1.6606e-01, -2.1464e+00, -6.8957e-01, -9.2513e-01, -2.5262e-01,
         -4.2558e-03, -7.1952e-01, -2.9928e-01, -1.0641e+00, -1.2072e+00,
         -3.4627e-02, -1.2238e-01, -4.7702e-01, -1.8247e-01, -5.4553e-01]])

In [ ]:
# soh quero ver a sequencia de transicao do que foi gerado (ignora o prompt original)

tamanho_prompt = len(prompt_input_ids[0])
print(tamanho_prompt)

# nosso prompt faz parte da resposta gerada!
print(output.sequences[0][:tamanho_prompt])

# Pegando soh o que foi gerado
generated_tokens = output.sequences[0][tamanho_prompt:]
print(generated_tokens)

print('| token id | score  | token str | prob % |')
print('|----------+--------+-----------+--------|')
for (token, score) in zip(generated_tokens, transitions[0]):
    if tiny_llama.tokenizer.decode(token) == '\n':
      continue

    print(f"| {token:8d} | {score.to('cpu').numpy():.3f} | {tiny_llama.tokenizer.decode(token):9s} | {np.exp(score.to('cpu').numpy()):.4f} |")

7
tensor([    1, 20628,   338,   263,  9560,  2462,   304])
tensor([  367, 18758, 29889,    13,    13,  6565,   344, 29871, 29906, 29901,
           13,  1576,  6575,   338,   528,  2827, 11785, 29892,   278, 17952,
          526, 23623, 29892,    13,  1576])
| token id | score  | token str | prob % |
|----------+--------+-----------+--------|
|      367 | -1.525 | be        | 0.2177 |
|    18758 | -0.583 | alive     | 0.5580 |
|    29889 | -0.959 | .         | 0.3833 |
|     6565 | -1.540 | Ver       | 0.2144 |
|      344 | -0.001 | se        | 0.9990 |
|    29871 | -0.023 |           | 0.9774 |
|    29906 | -1.078 | 2         | 0.3402 |
|    29901 | -0.062 | :         | 0.9398 |
|     1576 | -2.146 | The       | 0.1169 |
|     6575 | -0.690 | sun       | 0.5018 |
|      338 | -0.925 | is        | 0.3965 |
|      528 | -0.253 | sh        | 0.7768 |
|     2827 | -0.004 | ining     | 0.9958 |
|    11785 | -0.720 | bright    | 0.4870 |
|    29892 | -0.299 | ,         | 0.7413 |
|      27

In [ ]:
# Vendo o tempo de execucao para gerar um prompt

start = time.time()

output = tiny_llama(prompt, max_new_tokens=100)

end = time.time()

print(tiny_llama.model)
print(f'Tempo de execucao: {end - start:.2f} segundos')
print(output)
print(output[0]['generated_text'])

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [ ]:
# Lembrando da conversa

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = tiny_llama(prompt2, max_new_tokens=10)
print(output[0]['generated_text'])

# Ué, o que aconteceu? Alucinou?

What day is today?

Student: It's Monday.

--------------------------------------------------------------------------------
What day is tomorrow?

Tomorrow is Monday.

Inter


In [ ]:
# para se lembrar, precisamos alimentar ele com TUDO QUE FOI GERADO ANTERIORMENTE

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = tiny_llama(output[0]['generated_text'] + prompt2, max_new_tokens=30)
print(output[0]['generated_text'])

What day is today?

Student: It's Monday.

--------------------------------------------------------------------------------
What day is today?

Student: It's Monday.
. What day is tomorrow?

Student: It's Tuesday.
. What day is next week?

Student: It's Wednesday.


In [ ]:
# vamos carregar outro modelo (alucinou demais anteriormente)

# O código abaixo vai explodir a memória ram (pq ja temos o outro modelo carregado nela).
# deletando e limpando a memória

# import gc # garbage colector
# del tiny_llama
# gc.collect()
# torch.cuda.empty_cache()

# falcon = transformers.pipeline(
#     task='text-generation',
#     model='tiiuae/falcon-7b',
#     device=device
# )